In [8]:
import os
import tqdm
import torch
import numpy as np
from torchvision.models import resnet50

In [4]:
model = resnet50(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  
model.cuda()
model.eval()

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [17]:
image_folder = "processed_images/"
features = {}

# img_file = "{id}.npy"
for img_file in tqdm.tqdm(os.listdir(image_folder), total=len(os.listdir(image_folder)), desc="Extracting features"):
	book_id = img_file.split(".")[0]
	img_path = os.path.join(image_folder, img_file)
	img = np.load(img_path)
	img_tensor = torch.from_numpy(img).unsqueeze(0).cuda()
	
	with torch.no_grad():
		feature = model(img_tensor).squeeze()
		feature = feature.cpu().numpy()
		features[book_id] = feature

Extracting features: 100%|██████████| 13539/13539 [02:06<00:00, 106.73it/s]


In [18]:
import pickle
with open("image_features.pkl", "wb") as f:
    pickle.dump(features, f)